### Segmenting and clustering neighborhoods in Toronto
#### Project by Lydia Gómez

In [1]:
# setup
import pandas as pd
# set options for display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Import requests library
import requests

# Import folium library for maps
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


#### 1. Get the table from wikipedia and clean the data

In [2]:
# Import data from wikipedia
# define URL and take first element
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = pd.read_html(url)[0]

wiki.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# create a copy for working into it
df = wiki.copy()

In [4]:
# Remove rows with Not assigned Borough
b_remove = 'Not assigned'
remove_index = df[df['Borough'] == b_remove].index
df_clean = df.drop(remove_index)
df_clean.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
# Reset index for this dataset
df_clean.reset_index(inplace = True)
df_clean.drop(columns = ['index'], axis = 1, inplace = True)
df_clean.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
# Review if any postal code has more than one neighborhood
postal_codes = df_clean.groupby(['Postal Code']).count()
print('Number of duplicated boroughs: ', postal_codes['Borough'].value_counts())
print('Number of duplicated neighborhoods: ', postal_codes['Neighborhood'].value_counts())

Number of duplicated boroughs:  1    103
Name: Borough, dtype: int64
Number of duplicated neighborhoods:  1    103
Name: Neighborhood, dtype: int64


In [7]:
# Review if any Neighborhood has the Not assigned value
neigh = df_clean[df_clean['Neighborhood'] == b_remove]
neigh.head()

,Postal Code,Borough,Neighborhood


In [8]:
# Recheck in case the line above did not work
temp = df_clean['Borough'].where(df_clean['Neighborhood'] == b_remove)
print(temp.unique())

[nan]


In [9]:
# Print the shape of the dataframe
print(df_clean.shape)

(103, 3)


#### 2. Introduce the geolocation data into the table

In [10]:
# Install and import the library
!pip install geocoder
import geocoder

In [11]:
# Initialize variables 
lat_lng_coords = None
df_geo = df_clean.copy()

In [12]:
# Try the google geocoder, could not try the while loop as it got stuck into it
for postal_code in zip(df_geo['Postal Code']):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    print(postal_code, '...', g.latlng)

('M3A',) ... None
('M4A',) ... None
('M5A',) ... None
('M6A',) ... None
('M7A',) ... None
('M9A',) ... None
('M1B',) ... None
('M3B',) ... None
('M4B',) ... None
('M5B',) ... None
('M6B',) ... None
('M9B',) ... None
('M1C',) ... None
('M3C',) ... None
('M4C',) ... None
('M5C',) ... None
('M6C',) ... None
('M9C',) ... None
('M1E',) ... None
('M4E',) ... None
('M5E',) ... None
('M6E',) ... None
('M1G',) ... None
('M4G',) ... None
('M5G',) ... None
('M6G',) ... None
('M1H',) ... None
('M2H',) ... None
('M3H',) ... None
('M4H',) ... None
('M5H',) ... None
('M6H',) ... None
('M1J',) ... None
('M2J',) ... None
('M3J',) ... None
('M4J',) ... None
('M5J',) ... None
('M6J',) ... None
('M1K',) ... None
('M2K',) ... None
('M3K',) ... None
('M4K',) ... None
('M5K',) ... None
('M6K',) ... None
('M1L',) ... None
('M2L',) ... None
('M3L',) ... None
('M4L',) ... None
('M5L',) ... None
('M6L',) ... None
('M9L',) ... None
('M1M',) ... None
('M2M',) ... None
('M3M',) ... None
('M4M',) ... None
('M5M',) .

In [13]:
# It does not work so I will use the osm tool
for postal_code in zip(df_geo['Postal Code']):
    
    g = geocoder.osm('{}, Toronto, Ontario'.format(postal_code))
    print(postal_code, '...', g)

('M3A',) ... <[OK] Osm - Geocode [Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada]>
('M4A',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M5A',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M6A',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M7A',) ... <[OK] Osm - Geocode [Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada]>
('M9A',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M1B',) ... <[OK] Osm - Geocode [Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada]>
('M3B',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M4B',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M5B',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M6B',) ... <[ERROR - No results found] Osm - Geocode [empty]>
('M9B',) ... <[OK] Osm - Geocode [Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9B 1L3, Canada]>
('M1C',) ... <[OK] Osm - Geocode [Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada]>
('M3C',)

In [14]:
# As it does not work I will import the file with the data
import wget
url = 'http://cocl.us/Geospatial_data'
filename = 'Geospatial_Coordinates.csv'
wget.download(url, filename)

100% [................................................................................] 2891 / 2891

'Geospatial_Coordinates (2).csv'

In [15]:
# Read file into dataframe
df_coord = pd.read_csv(filename)
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
# merge dataframes
df_geo = df_geo.merge(df_coord, on = 'Postal Code')
df_geo.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [17]:
# Print shape in order to check whether the merge included duplicated
print(df_geo.shape)

(103, 5)


#### 3. Cluster the neighborhoods

In [18]:
# First create a map to visualize all the neighborhoods in the dataset

latitude = df_geo['Latitude'][2]
longitude = df_geo['Longitude'][2]
# Use the M5A postal code data to center the map
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add markers to the map
for lat, long, borough, neigh in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighborhood']):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 4,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.5,
        parse_html = False).add_to(map_toronto)

map_toronto

In [19]:
# Define foursquare credentials and details
CLIENT_ID = '0JSTHXGCUEZQN3BTUK2BGK13FOWCNVBUM35HUA54M5X0ZHWP' # your Foursquare ID
CLIENT_SECRET = 'SM4AZKGXNPXAVW5KVWITBSJNLBLN4SUFXKC0Y245STCD41U2' # your Foursquare Secret
VERSION = '20200702' # Foursquare API version
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0JSTHXGCUEZQN3BTUK2BGK13FOWCNVBUM35HUA54M5X0ZHWP
CLIENT_SECRET:SM4AZKGXNPXAVW5KVWITBSJNLBLN4SUFXKC0Y245STCD41U2


In [20]:
# Create a function that get all near venues for all the neighborhoods

def getNearbyVenues(codes, boroughs, names, latitudes, longitudes, radius = 500):
    venues_list = []
    for code, borough, name, lat, long in zip(codes, boroughs, names, latitudes, longitudes):
        # Create API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            long, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # gather only relevant information for each venue
        venues_list.append([(code, borough, name, lat, long, 
                             v['venue']['name'], v['venue']['location']['lat'], 
                             v['venue']['location']['lng'], v['venue']['categories'][0]['name']) 
                            for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 'Borough', 'Neighborhood', 'Neighborhood lat', 'Neighborhood long',
                            'Venue', 'Venue lat', 'Venue long', 'Venue category']

    return(nearby_venues)


In [21]:
# Let's run the function for our neighborhodds in Toronto
toronto_venues = getNearbyVenues(codes = df_geo['Postal Code'],
                                 boroughs = df_geo['Borough'],
                                 names = df_geo['Neighborhood'], 
                                 latitudes = df_geo['Latitude'], 
                                 longitudes = df_geo['Longitude'])

toronto_venues.head()

,Postal Code,Borough,Neighborhood,Neighborhood lat,Neighborhood long,Venue,Venue lat,Venue long,Venue category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,North York,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,North York,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,North York,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [22]:
# review how many venues were returned for each neighborhood (as done with the value_counts)
toronto_venues.groupby('Neighborhood').count()

,Postal Code,Borough,Neighborhood lat,Neighborhood long,Venue,Venue lat,Venue long,Venue category
Neighborhood,,,,,,,,
Agincourt,5,5,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25,25,25
Berczy Park,58,58,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17,17,17


In [23]:
# check how many categories for the venues we have gathered
print('There are {} unique venue categories'.format(len(toronto_venues['Venue category'].unique())))

There are 273 unique venue categories


In [24]:
# Let's analyze each neighborhood and use one hot encoding to transform the dataframe
toronto_onehot = pd.get_dummies(toronto_venues[['Venue category']], prefix = '', prefix_sep = '')

# add neighborhood column to the one hot dataframe
toronto_onehot['Neigh'] = toronto_venues['Neighborhood']

# move neighborhood to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neigh,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
# Let's examine the dataset and check all venues categories are translated
print(toronto_onehot.shape)

(2118, 274)


In [26]:
# Now let's group by neighborhood and calculate the mean of the appearences of each category
toronto_g = toronto_onehot.groupby('Neigh').mean().reset_index()
toronto_g.head(10)

,Neigh,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0

In [27]:
# review the new size
print('The new size of the dataframe is: ', toronto_g.shape)

The new size of the dataframe is:  (96, 274)


In [28]:
# Let's put the top 10 venues for each neighborhood in a dataframe
# first, create a function to sort descending

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return(row_categories_sorted.index.values[0:num_top_venues])
    
# Second, create the dataframe and displacy the 10 most common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
# create columns according with numbers of top venues
columns = ['Neigh']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} most common venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))

# create a new dataframe        
neigh_venues_sorted = pd.DataFrame(columns = columns)
neigh_venues_sorted['Neigh'] = toronto_g['Neigh']

for ind in np.arange(toronto_g.shape[0]):
    neigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_g.iloc[ind, :], num_top_venues)

neigh_venues_sorted   

,Neigh,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
1,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Coffee Shop,Athletics & Sports,Sandwich Place,Pub,Diner,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Sandwich Place,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Ice Cream Shop,Supermarket,Fried Chicken Joint,Deli / Bodega
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Juice Bar,Butcher,Café,Indian Restaurant
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Beer Bar,Bakery,Café,Seafood Restaurant,Cheese Shop,Breakfast Spot,Jazz Club
6,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
8,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Garden Center,Brewery,Light Rail Station,Spa,Farmers Market,Fast Food Restaurant,Burrito Place,Restaurant,Butcher
9,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry


In [29]:
# Clusster the neighborhoods by running k means and cluser the neighborhoods in 5 clusters
# set number of clusters
kclusters = 4

toronto_g_clustering = toronto_g.drop('Neigh', axis = 1)

# run K means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0)
kmeans.fit(toronto_g_clustering)

# review cluster labels
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
# Check 4 clusters and review the number of neighborhoods in each cluster
np.unique(kmeans.labels_, return_counts = True)

(array([0, 1, 2, 3]), array([ 8, 85,  2,  1], dtype=int64))

In [33]:
# Let's create a dataframe with the ten most common venues, the neighborhoods and the cluster
neigh_venues_sorted.insert(0, 'Cluster labels', kmeans.labels_)

toronto_merged = df_geo

# merge toronto data with toronto venues information, as not all neighborhoods have venues information I have to do the
# merge inner
toronto_merged = toronto_merged.join(neigh_venues_sorted.set_index('Neigh'), on = 'Neighborhood', how = 'inner')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster labels,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Coffee Shop,French Restaurant,Hockey Arena,Intersection,Pizza Place,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Shoe Store,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Accessories Store,Sporting Goods Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Event Space,Coffee Shop,Vietnamese Restaurant,Construction & Landscaping
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place


In [34]:
# Let's visualize the results

map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

# set color scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker([lat, long],
                       radius = 5,
                       popup = label,
                       color = rainbow[cluster-1],
                       fill = True,
                       fill_color = rainbow[cluster-1],
                       fill_opacity = 0.6).add_to(map_clusters)

map_clusters


#### 4. Examine clusters

In [35]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster labels'] == 0, toronto_merged.columns[[2] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Parkwoods,Park,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
21,Caledonia-Fairbanks,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
35,"East Toronto, Broadview North (Old East York)",Park,Coffee Shop,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
64,Weston,Convenience Store,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
66,York Mills West,Convenience Store,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
85,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
91,Rosedale,Playground,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
98,"The Kingsway, Montgomery Road, Old Mill North",Smoke Shop,River,College Stadium,Colombian Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,College Rec Center,Drugstore


Cluster 1: this cluster has neighborhoods that have within the top 10 venues several tipes of restaurants (Sim Sum, Doner, Coffee Shops, Diners, Dessert shops) and also sport venues such as parks, dog runs or Yoga Studies and also have stores (discount, department store)

In [36]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster labels'] == 1, toronto_merged.columns[[2] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
1,Victoria Village,Coffee Shop,French Restaurant,Hockey Arena,Intersection,Pizza Place,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant
2,"Regent Park, Harbourfront",Park,Bakery,Pub,Breakfast Spot,Café,Theater,Cosmetics Shop,Shoe Store,Restaurant
3,"Lawrence Manor, Lawrence Heights",Accessories Store,Sporting Goods Shop,Boutique,Miscellaneous Shop,Furniture / Home Store,Event Space,Coffee Shop,Vietnamese Restaurant,Construction & Landscaping
4,"Queen's Park, Ontario Provincial Government",Diner,Sushi Restaurant,Yoga Studio,College Auditorium,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place
6,"Malvern, Rouge",Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Harbor / Marina
7,Don Mills,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Asian Restaurant,Sporting Goods Shop,Italian Restaurant,Dim Sum Restaurant,Café
13,Don Mills,Restaurant,Japanese Restaurant,Coffee Shop,Beer Store,Asian Restaurant,Sporting Goods Shop,Italian Restaurant,Dim Sum Restaurant,Café
8,"Parkview Hill, Woodbine Gardens",Gym / Fitness Center,Gastropub,Pharmacy,Café,Breakfast Spot,Bank,Intersection,Athletics & Sports,Fast Food Restaurant
9,"Garden District, Ryerson",Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Italian Restaurant,Pizza Place,Bakery
10,Glencairn,Asian Restaurant,Pub,Japanese Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


CLuster 2: This cluster is the grouping of all the neighborhoods that cannot be clearly assigned to a cluster and does not have a certain pattern

In [37]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster labels'] == 2, toronto_merged.columns[[2] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
57,"Humberlea, Emery",Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Fast Food Restaurant
101,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Fast Food Restaurant


CLuster 3: In this cluster the main venue is a Yoga Studio, followed by restaurants and a discount store and distribution center. It could be a new neighborhood with young people

In [38]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster labels'] == 3, toronto_merged.columns[[2] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
83,"Moore Park, Summerhill East",Yoga Studio,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant


CLuster 4: Is similar to cluster 3 but in this case it has a deli as the second most popular venue and a dessert shop as the third one

In [39]:
# Cluster 5
toronto_merged.loc[toronto_merged['Cluster labels'] == 4, toronto_merged.columns[[2] + list(range(7, toronto_merged.shape[1]))]]

,Neighborhood,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
